### Introduction

In this notebook all data concerning the harvesting of tweets is present.
The output of this notebook is a csv containing the necessary data for the analyses in the later notebooks.

In [16]:
import pandas as pd
import seaborn as sns
import json
import tweepy
import os

import re
import emoji

import csv
import time

import dataset #https://dataset.readthedocs.io/en/latest/api.html
from datafreeze import freeze

In [17]:
# Setup connection with the API, add keys 

# Key
Key1 = ""

# Secret Key
Key2 = "" 

# Generate the access tokens in developer account: https://developer.twitter.com/en/portal/dashboard
# See screenshot below

# Access token
ACCESS_TOKEN = ""

# Access token secret
ACCESS_TOKEN_SECRET = ""

In [18]:
# Set up and test API

# Authorize 
auth = tweepy.OAuthHandler(Key1, Key2)

# Authorize with access token
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

# Test authorization
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
# Set Global variables for harvesting of tweets

# Database Variables
CONNECTION_STRING = "sqlite:///tweets_historic_2.db"
CSV_NAME = "tweets_historic_full_text.csv"
TABLE_NAME = "tweets_historic"

# API Rate Limit Variables
MAX_REQUEST = 1000
PAUSE_TIME = 960 # 16 minutes

# Open Database
db2 = dataset.connect(CONNECTION_STRING)

In [ ]:
def clean_text(text):
    '''clean text of links, special characters, @mentions, 
    double spaces, lowercase everthing'''
    cleaned = re.sub(r"(?:\@|https?\://)\S+", "", text)
    cleaned = ' '.join(cleaned.split())
    cleaned = cleaned.lower()
    return(cleaned)

def create_db_historic_tweets(number, NTweets, last_max_id):
    '''request tweets from API and add them to the 
    DB in the right format'''
    
    #get tweets with query
    statuses = tweepy.Cursor(api.search, q='(covid OR corona) -filter:retweets', 
                             tweet_mode = 'extended', lang = "nl", max_id = last_max_id).items(number)
    
    #save max_id to be able to get increasingly old tweets each cycle
    max_status_id = 0
    
    #main loop for saving statusses to DB 
    for status in statuses:
        description = status.user.description
        loc = status.user.location
        coords = status.coordinates
        geo = status.geo
        name = status.user.screen_name
        user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
        retweets = status.retweet_count
        bg_color = status.user.profile_background_color
        text = status.full_text

        if geo is not None:
            geo = json.dumps(geo)

        if coords is not None:
            coords = json.dumps(coords)   

        table = db2[TABLE_NAME]
        max_status_id = id_str  

        text = clean_text(text)
        loc = clean_text(loc)
        try: 
            table.insert(dict(
                user_description=description,
                user_location=loc,
                coordinates=coords,
                text=text,
                geo=geo,
                user_name=name,
                user_created=user_created,
                user_followers=followers,
                id_str=id_str,
                created=created,
                retweet_count=retweets,
                user_bg_color=bg_color,
            ))
        except:
            print("error")
    print(NTweets + MAX_REQUEST, "tweets at Time: %s" % time.ctime())
    return NTweets + MAX_REQUEST, max_status_id


def night_runner():
    '''Main loop for requesting tweets, pauses after each request'''
    # reset below variables with last tweet-id each time the API disconnects
    NTweets = 57000  
    last_max_id = "1382689495179485191"
    while(True):
        NTweets, last_max_id = create_db_historic_tweets(MAX_REQUEST, NTweets, last_max_id)
        time.sleep(PAUSE_TIME)
        


In [ ]:
# Run main loop to collect tweets in DB 
night_runner()

In [ ]:
# Connect to database 
db2 = dataset.connect("sqlite:///tweets_historic.db")

# Store the resulting DB in a csv
result = db2[TABLE_NAME].all()

# Export result to csv 
freeze(result, format='csv', filename=CSV_NAME)

In [22]:
# Below is some extra code to test the process and output

In [ ]:
# Read in csv and print length 
data = pd.read_csv(CSV_NAME)
len(data)

In [ ]:
# display data 
pd.set_option('display.max_colwidth', None)
data

In [ ]:
data["text"].nunique()

### Discussion

#### (Design)Choices
Tweepy: Because of the extensive and easy (or easier) to understand documentation, compared to for example twython, we decided to use Tweepy to handle the connection with the API 

Database: We decided to add the tweets directly to a DB and output the DB to a csv afterwards. This way we could add tweets to the database one-by-one, and if the API was interupted we could easily restart it without causing problems with the already harvested data.


#### Problems
API Documentation was very complex, and often outated because of the recently implemented V2 API of twitter. It took a long time to get the API running. 

On the first run we collected 15.000 tweets, but found out afterwards that we did not collect the full text tweets. This happened because we only tested the query for full text in the Live API, and assumed it would also work in the REST API. 
